In [ ]:
def train_deep_cbn(dataset_path, target_col, smiles_col):
    import os
    import numpy as np
    import pandas as pd
    import tensorflow as tf
    import keras
    from sklearn.model_selection import train_test_split
    from sklearn.utils.class_weight import compute_class_weight
    from tensorflow.keras import optimizers, layers, metrics
    from tensorflow.keras.layers import (
        Input, Dense, Dropout, BatchNormalization, Conv1D,
        GlobalAveragePooling1D, Lambda, Activation, Concatenate
    )
    from tensorflow.keras.models import Model
    from tensorflow.keras.utils import to_categorical
    from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping

    # === Load Dataset ===
    data = pd.read_csv(dataset_path)
    smiles = data[smiles_col]
    labels = data[target_col]

    # === Tokenization ===
    smiles_dict = {
        "#": 29, "%": 30, ")": 31, "(": 1, "+": 32, "-": 33, "/": 34, ".": 2,
        "1": 35, "0": 3, "3": 36, "2": 4, "5": 37, "4": 5, "7": 38, "6": 6,
        "9": 39, "8": 7, "=": 40, "A": 41, "@": 8, "C": 42, "B": 9, "E": 43,
        "D": 10, "G": 44, "F": 11, "I": 45, "H": 12, "K": 46, "M": 47, "L": 13,
        "O": 48, "N": 14, "P": 15, "S": 49, "R": 16, "U": 50, "T": 17, "W": 51,
        "V": 18, "Y": 52, "[": 53, "Z": 19, "]": 54, "\": 20, "a": 55, "c": 56,
        "b": 21, "e": 57, "d": 22, "g": 58, "f": 23, "i": 59, "h": 24, "m": 60,
        "l": 25, "o": 61, "n": 26, "s": 62, "r": 27, "u": 63, "t": 28, "y": 64,
        " ": 65, ":": 66, ",": 67, "p": 68, "j": 69, "*": 70
    }

    def label_smiles(line, max_len, smi_ch_ind):
        x = np.zeros(max_len, dtype=int)
        for i, ch in enumerate(line[:max_len]):
            if ch != "
" and ch in smi_ch_ind:
                x[i] = smi_ch_ind[ch]
        return x

    XD = np.array([label_smiles(str(s), 100, smiles_dict) for s in smiles])
    labels = labels.values
    XD = to_categorical(XD, num_classes=71)

    # === Feature model ===
    XDinput = Input(shape=(100, 71), name='XDinput')
    encode_smiles = Conv1D(filters=64, kernel_size=2, activation='relu', padding='valid', strides=1)(XDinput)
    encode_smiles = Conv1D(filters=64, kernel_size=4, activation='relu', padding='valid', strides=1)(encode_smiles)
    encode_smiles = Conv1D(filters=128, kernel_size=4, activation='relu', padding='valid', strides=1)(encode_smiles)
    model_feature = Model(inputs=XDinput, outputs=encode_smiles, name='model_feature')

    input_extracted_feature = Input(shape=(93, 128))
    FC1 = Dense(512, activation='relu')(input_extracted_feature)
    FC1 = BatchNormalization()(FC1)
    FC2 = Dropout(0.1)(FC1)
    FC3 = Dense(256, activation='relu')(FC2)
    FC3 = BatchNormalization()(FC3)
    FC4 = Dropout(0.1)(FC3)
    FC5 = Dense(64, activation='relu')(FC4)
    predictions = Dense(2, activation='softmax')(FC5)
    model_pred = Model(inputs=input_extracted_feature, outputs=predictions)

    interaction_input = XDinput
    encoded_features = model_feature(interaction_input)
    predicted_output = model_pred(encoded_features)
    pooled_output = GlobalAveragePooling1D()(predicted_output)
    interactionModel = Model(inputs=interaction_input, outputs=pooled_output, name='interactionModel')

    class DropPath(layers.Layer):
        def __init__(self, drop_prob=0.):
            super().__init__()
            self.drop_prob = drop_prob
        def call(self, x, training=None):
            if (not training) or self.drop_prob == 0.:
                return x
            keep_prob = 1 - self.drop_prob
            random_tensor = keep_prob
            random_tensor += tf.random.uniform(tf.shape(x), dtype=x.dtype)
            binary_tensor = tf.floor(random_tensor)
            return tf.divide(x, keep_prob) * binary_tensor

    class TopkRouting(layers.Layer):
        def __init__(self, qk_dim, topk=16, qk_scale=None, param_routing=False, diff_routing=False):
            super().__init__()
            self.topk = topk
            self.qk_dim = qk_dim
            self.scale = qk_scale if qk_scale is not None else qk_dim**-0.5
            self.diff_routing = diff_routing
            if param_routing:
                self.emb = layers.Dense(qk_dim)
            else:
                self.emb = lambda x: x
            self.routing_act = lambda x, axis: tf.nn.softmax(x, axis=axis)
        def call(self, query, key, training=None):
            if not self.diff_routing:
                query = tf.stop_gradient(query)
                key = tf.stop_gradient(key)
            query_hat = self.emb(query)
            key_hat = self.emb(key)
            attn_logit = tf.einsum('bnc,bqc->bnq', query_hat*self.scale, key_hat)
            topk_attn_logit, topk_index = tf.math.top_k(attn_logit, k=self.topk, sorted=True)
            r_weight = self.routing_act(topk_attn_logit, axis=-1)
            return r_weight, topk_index

    def tf_gather_kv(kv, r_idx):
        n = tf.shape(kv)[0]
        p2 = tf.shape(kv)[1]
        c_kv = tf.shape(kv)[2]
        topk = tf.shape(r_idx)[2]
        batch_idx = tf.reshape(tf.range(n), [n, 1, 1])
        batch_idx = tf.tile(batch_idx, [1, p2, topk])
        p2_idx = tf.reshape(tf.range(p2), [1, p2, 1])
        p2_idx = tf.tile(p2_idx, [n, 1, topk])
        gather_indices = tf.stack([batch_idx, p2_idx, r_idx], axis=-1)
        gathered = tf.gather_nd(kv, gather_indices)
        return gathered

    class KVGather(layers.Layer):
        def __init__(self, mul_weight='none'):
            super().__init__()
            assert mul_weight in ['none', 'soft', 'hard']
            self.mul_weight = mul_weight
        def call(self, r_idx, r_weight, kv, training=None):
            topk_kv = tf_gather_kv(kv, r_idx)
            if self.mul_weight == 'soft':
                r_weight_exp = tf.expand_dims(tf.expand_dims(r_weight, -1), -1)
                topk_kv = topk_kv * r_weight_exp
            return topk_kv

    class QKVLinear(layers.Layer):
        def __init__(self, dim, qk_dim, bias=True):
            super().__init__()
            self.dim = dim
            self.qk_dim = qk_dim
            self.qkv = layers.Dense(qk_dim + qk_dim + dim, use_bias=bias)
        def call(self, x, training=None):
            qkv = self.qkv(x)
            q, kv = tf.split(qkv, [self.qk_dim, self.qk_dim + self.dim], axis=-1)
            return q, kv

    class BiLevelRoutingAttention(layers.Layer):
        def __init__(self, dim, n_win=7, num_heads=8, qk_dim=None, qk_scale=None,
                     kv_per_win=4, kv_downsample_ratio=4, kv_downsample_mode='identity',
                     topk=4, param_attention="qkvo", param_routing=False, diff_routing=False, soft_routing=False,
                     side_dwconv=3, auto_pad=True):
            super().__init__()
            self.dim = dim
            self.n_win = n_win
            self.num_heads = num_heads
            self.qk_dim = qk_dim if qk_dim is not None else dim
            self.scale = qk_scale if qk_scale is not None else self.qk_dim**-0.5
            self.topk = topk
            self.param_routing = param_routing
            self.diff_routing = diff_routing
            self.soft_routing = soft_routing
            self.auto_pad = auto_pad
            if side_dwconv > 0:
                self.lepe = layers.Conv1D(dim, kernel_size=side_dwconv, padding='same', activation='relu')
            else:
                self.lepe = lambda x: tf.zeros_like(x)
            self.router = TopkRouting(qk_dim=self.qk_dim, topk=self.topk, qk_scale=self.scale,
                                      param_routing=self.param_routing, diff_routing=self.diff_routing)
            mul_weight = 'none'
            if self.soft_routing:
                mul_weight = 'soft'
            self.kv_gather = KVGather(mul_weight=mul_weight)
            if param_attention in ['qkvo', 'qkv']:
                self.qkv = QKVLinear(self.dim, self.qk_dim)
                if param_attention == 'qkvo':
                    self.wo = layers.Dense(self.dim)
                else:
                    self.wo = lambda x: x
            else:
                raise ValueError("Unsupported param_attention mode")
            self.attn_act = lambda x: tf.nn.softmax(x, axis=-1)
            self.kv_down = lambda x: x
        def call(self, x, training=None, ret_attn_mask=False):
            if ret_attn_mask:
                return x, None, None, None
            else:
                return x

    class Attention(layers.Layer):
        def __init__(self, dim, num_heads=8, qkv_bias=False, qk_scale=None, attn_drop=0., proj_drop=0.):
            super().__init__()
            self.num_heads = num_heads
            head_dim = dim // num_heads
            self.scale = qk_scale if qk_scale is not None else head_dim**-0.5
        def call(self, x, training=None):
            return x

    class AttentionLePE(layers.Layer):
        def __init__(self, dim, num_heads=8, qkv_bias=False, qk_scale=None, attn_drop=0., proj_drop=0., side_dwconv=3):
            super().__init__()
            self.num_heads = num_heads
            head_dim = dim // num_heads
            self.scale = qk_scale if qk_scale is not None else head_dim**-0.5
            self.side_dwconv = layers.Conv1D(dim, kernel_size=side_dwconv, padding='same', activation='relu')
        def call(self, x, training=None):
            return x

    class Mlp(layers.Layer):
        def __init__(self, in_features, hidden_features=None, out_features=None, drop=0.):
            super().__init__()
            out_features = out_features or in_features
            hidden_features = hidden_features or in_features
            self.net = keras.Sequential([
                layers.Dense(hidden_features),
                layers.Activation('gelu'),
                layers.Dropout(rate=drop),
                layers.Dense(out_features),
                layers.Dropout(rate=drop)
            ])
        def call(self, x, training=True):
            return self.net(x, training=training)

    class Block(layers.Layer):
        def __init__(self, dim, drop_path=0.1, layer_scale_init_value=-1,
                     num_heads=8, n_win=7, qk_dim=128, qk_scale=None,
                     kv_per_win=8, kv_downsample_ratio=1, kv_downsample_mode='identity',
                     topk=8, param_attention="qkvo", param_routing=True, diff_routing=False, soft_routing=True,
                     mlp_ratio=4, mlp_dwconv=False, side_dwconv=5, before_attn_dwconv=3, pre_norm=True, auto_pad=True):
            super().__init__()
            qk_dim = qk_dim or dim
            if before_attn_dwconv > 0:
                self.pos_embed = layers.Conv1D(dim, kernel_size=before_attn_dwconv, padding='same', activation='relu')
            else:
                self.pos_embed = lambda x: tf.zeros_like(x)
            self.norm1 = layers.LayerNormalization(epsilon=1e-6)
            if topk > 0:
                self.attn = BiLevelRoutingAttention(dim=dim, num_heads=num_heads, n_win=n_win, qk_dim=qk_dim,
                                                   qk_scale=qk_scale, kv_per_win=kv_per_win, kv_downsample_ratio=kv_downsample_ratio,
                                                   kv_downsample_mode=kv_downsample_mode, topk=topk, param_attention=param_attention,
                                                   param_routing=param_routing, diff_routing=diff_routing, soft_routing=soft_routing,
                                                   side_dwconv=side_dwconv, auto_pad=auto_pad)
            elif topk == -1:
                self.attn = Attention(dim=dim, num_heads=num_heads, qk_scale=qk_scale)
            elif topk == -2:
                self.attn = AttentionLePE(dim=dim, num_heads=num_heads, qk_scale=qk_scale, side_dwconv=side_dwconv)
            else:
                self.attn = keras.Sequential([
                    layers.Dense(dim),
                    layers.Conv1D(dim, kernel_size=5, padding='same', activation='relu'),
                    layers.Dense(dim)
                ])
            self.norm2 = layers.LayerNormalization(epsilon=1e-6)
            mlp_hidden_dim = int(mlp_ratio * dim)
            mlp_layers = [layers.Dense(mlp_hidden_dim)]
            if mlp_dwconv:
                mlp_layers.append(layers.Conv1D(mlp_hidden_dim, kernel_size=3, padding='same', activation='relu'))
            mlp_layers.append(layers.Activation('gelu'))
            mlp_layers.append(layers.Dense(dim))
            mlp_layers.insert(1, layers.Dropout(0.2))
            mlp_layers.append(layers.Dropout(0.2))
            self.mlp = keras.Sequential(mlp_layers)
            self.drop_path = DropPath(drop_path) if drop_path > 0. else layers.Lambda(lambda x: x)
            if layer_scale_init_value > 0:
                self.use_layer_scale = True
                self.gamma1 = self.add_weight(shape=(dim,), initializer=tf.keras.initializers.Constant(layer_scale_init_value), trainable=True)
                self.gamma2 = self.add_weight(shape=(dim,), initializer=tf.keras.initializers.Constant(layer_scale_init_value), trainable=True)
            else:
                self.use_layer_scale = False
        def call(self, x, training=None):
            input_x = x
            x = self.pos_embed(x)
            x = self.norm1(x)
            attn = self.attn(x, training=training)
            if self.use_layer_scale:
                attn = self.gamma1 * attn
            x = input_x + self.drop_path(attn, training=training)
            input_x = x
            x = self.norm2(x)
            mlp_output = self.mlp(x, training=training)
            if self.use_layer_scale:
                mlp_output = self.gamma2 * mlp_output
            x = input_x + self.drop_path(mlp_output, training=training)
            return x

    dim = 128
    num_heads = 8

    input_phaz2 = Input(shape=(93, 128))
    processed_input = input_phaz2
    transformer_block1 = Block(dim=dim, drop_path=0.2, num_heads=num_heads, topk=16, mlp_ratio=3)
    transformer_block2 = Block(dim=dim, drop_path=0.2, num_heads=num_heads, topk=16, mlp_ratio=3)
    transformer_output1 = transformer_block1(processed_input)
    transformer_output1 = Lambda(lambda x: tf.norm(x, axis=-1, keepdims=True))(transformer_output1)
    transformer_output2 = transformer_block2(processed_input)
    transformer_output2 = Lambda(lambda x: tf.norm(x, axis=-1, keepdims=True))(transformer_output2)
    combined_outputs = Concatenate(axis=-1)([transformer_output1, transformer_output2])
    pooled_outputs = GlobalAveragePooling1D()(combined_outputs)
    difference = Lambda(lambda x: tf.expand_dims(x[:, 0] - x[:, 1], axis=-1), name='difference')(pooled_outputs)
    condition_2 = Activation('sigmoid', name='activation_11')(difference)
    model_feature.trainable = False
    model_phaz2 = Model(inputs=input_phaz2, outputs=condition_2)

    new_transformer_block = Block(dim=dim, drop_path=0.2, num_heads=num_heads, topk=16, mlp_ratio=3)
    new_transformer_block2 = Block(dim=dim, drop_path=0.2, num_heads=num_heads, topk=16, mlp_ratio=3)
    XDinput_phaz3 = Input(shape=(100, 71))
    model_feature_output = model_feature(XDinput_phaz3)
    transformer_output1_phaz3 = new_transformer_block(model_feature_output)
    transformer_output1_phaz3 = Lambda(lambda x: tf.norm(x, axis=-1, keepdims=True))(transformer_output1_phaz3)
    transformer_output2_phaz3 = new_transformer_block2(model_feature_output)
    transformer_output2_phaz3 = Lambda(lambda x: tf.norm(x, axis=-1, keepdims=True))(transformer_output2_phaz3)
    concatenated_phaz3 = Concatenate(axis=-1)([transformer_output1_phaz3, transformer_output2_phaz3])
    model_feature.trainable = False
    new_transformer_block.trainable = False
    new_transformer_block2.trainable = False
    pooled_phaz3 = GlobalAveragePooling1D()(concatenated_phaz3)
    FC1_phaz3 = Dense(512, activation='relu')(pooled_phaz3)
    FC1_phaz3 = BatchNormalization()(FC1_phaz3)
    FC2_phaz3 = Dropout(0.1)(FC1_phaz3)
    FC3_phaz3 = Dense(256, activation='relu')(FC2_phaz3)
    FC3_phaz3 = BatchNormalization()(FC3_phaz3)
    FC4_phaz3 = Dropout(0.1)(FC3_phaz3)
    FC5_phaz3 = Dense(64, activation='relu')(FC4_phaz3)
    predictions_phaz3 = Dense(2, activation='softmax')(FC5_phaz3)
    model_phaz3 = Model(inputs=XDinput_phaz3, outputs=predictions_phaz3)

    METRICS_BINARY = [
        metrics.BinaryAccuracy(name='accuracy'),
        metrics.AUC(name='auc'),
    ]
    METRICS_CATEGORICAL = [
        metrics.CategoricalAccuracy(name='accuracy'),
        metrics.Precision(name='precision'),
        metrics.Recall(name='recall'),
        metrics.AUC(name='auc'),
        metrics.F1Score(average='macro', name='f1_score')
    ]

    XD_np = np.array(XD)
    labels_np = np.array(labels)
    index = np.where(~np.isnan(labels_np))
    labels_np, XD_np = labels_np[index[0]], XD_np[index[0], :]

    X_train, X_test, y_train, y_test = train_test_split(
        XD_np, labels_np, test_size=0.2, stratify=labels_np, random_state=9
    )

    y_train_cat = to_categorical(y_train, num_classes=2)
    y_test_cat = to_categorical(y_test, num_classes=2)

    class_weights_array = compute_class_weight(
        class_weight='balanced',
        classes=np.unique(y_train),
        y=y_train.ravel()
    )
    class_weight = {0: class_weights_array[0], 1: class_weights_array[1]}

    opt = optimizers.Adam(learning_rate=0.0001)
    interactionModel.compile(optimizer=opt, loss='categorical_crossentropy', metrics=METRICS_CATEGORICAL)

    early_stopping = EarlyStopping(monitor='loss', patience=30, verbose=1, mode='min', restore_best_weights=True)

    interactionModel.fit(
        X_train, y_train_cat,
        batch_size=256, epochs=100, class_weight=class_weight,
        callbacks=[early_stopping], verbose=1
    )
    test_eval_phase1 = interactionModel.evaluate(X_test, y_test_cat, verbose=1)

    feature_train = model_feature.predict(X_train)
    feature_test = model_feature.predict(X_test)

    opt_phaz2 = optimizers.Adam(learning_rate=0.001)
    model_phaz2.compile(optimizer=opt_phaz2, loss='binary_crossentropy', metrics=METRICS_BINARY)
    model_phaz2.fit(
        feature_train, y_train,
        batch_size=256, epochs=100, class_weight=class_weight,
        callbacks=[early_stopping], verbose=1
    )

    new_transformer_block.set_weights(transformer_block1.get_weights())
    new_transformer_block2.set_weights(transformer_block2.get_weights())

    opt_phaz3 = optimizers.Adam(learning_rate=0.0001)
    model_phaz3.compile(optimizer=opt_phaz3, loss='categorical_crossentropy', metrics=METRICS_CATEGORICAL)
    model_phaz3.fit(
        X_train, y_train_cat,
        batch_size=256, epochs=100, class_weight=class_weight,
        callbacks=[early_stopping], verbose=1
    )
    train_eval_phase3 = model_phaz3.evaluate(X_train, y_train_cat, verbose=1)
    test_eval_phase3 = model_phaz3.evaluate(X_test, y_test_cat, verbose=1)

    def format_results(phase1_result, phase3_train_result, phase3_test_result):
        phase1_metrics = ['loss', 'accuracy', 'precision', 'recall', 'auc', 'f1_score']
        phase3_metrics = ['loss', 'accuracy', 'precision', 'recall', 'auc', 'f1_score']
        phase1_test_df = pd.DataFrame([phase1_result], columns=phase1_metrics)
        phase3_train_df = pd.DataFrame([phase3_train_result], columns=phase3_metrics)
        phase3_test_df = pd.DataFrame([phase3_test_result], columns=phase3_metrics)
        print("
=== Phase 1: Test Evaluation Results ===")
        print(phase1_test_df.to_string(index=False, float_format=lambda x: f"{x:.4f}"))
        print("
=== Phase 3: Train Evaluation Results ===")
        print(phase3_train_df.to_string(index=False, float_format=lambda x: f"{x:.4f}"))
        print("
=== Phase 3: Test Evaluation Results ===")
        print(phase3_test_df.to_string(index=False, float_format=lambda x: f"{x:.4f}"))

    format_results(test_eval_phase1, train_eval_phase3, test_eval_phase3)
    return interactionModel, model_phaz2, model_phaz3
